- assign zone_id to each row in the subway dataset
- combine the subway and taxi datasets.
- running this scrip generates a csv file for the combined dataframes.


In [122]:
import json
from shapely.geometry import shape, Point
import os
!pip install python-dotenv
from dotenv import load_dotenv
import geopandas as gpd
import pandas as pd
import itertools
import numpy as np
import seaborn as sns
!pip install optuna
import optuna
import folium
from folium.plugins import HeatMap
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import xgboost as xgb
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [2]:
load_dotenv('/content/.env')

True

In [ ]:
# Download the file from Google Drive using the file ID
!gdown {os.getenv('taxi-geojason')}
!gdown {os.getenv('cleaned-subway-data')}
!gdown {os.getenv('cleaned-taxi-data')}

In [160]:
#load the taxi dataset into a df
taxi_df = pd.read_csv('/content/combined_taxi_df.csv')

taxi_df.head()

,datetime_formatted,hour,day_of_week,week,month,day_of_month,year_month,zone,passenger_count
0,2021-01-01 00:00:00,0,4,53,0,1,2021-01,4,4
1,2021-01-01 00:00:00,0,4,53,0,1,2021-01,13,3
2,2021-01-01 00:00:00,0,4,53,0,1,2021-01,24,3
3,2021-01-01 00:00:00,0,4,53,0,1,2021-01,41,12
4,2021-01-01 00:00:00,0,4,53,0,1,2021-01,42,2


In [161]:
#drop all columns except of datetime_formated and passenger_count and zone
taxi_df = taxi_df[["datetime_formatted", "zone", "passenger_count"]].reset_index(drop=True)

In [162]:
#rename datetime_formatted to transit_timestamp and zone to zone_id
taxi_df.rename(columns={'datetime_formatted': 'transit_timestamp', 'zone': 'zone_id' }, inplace=True)

In [163]:
taxi_df.head()

,transit_timestamp,zone_id,passenger_count
0,2021-01-01 00:00:00,4,4
1,2021-01-01 00:00:00,13,3
2,2021-01-01 00:00:00,24,3
3,2021-01-01 00:00:00,41,12
4,2021-01-01 00:00:00,42,2


In [164]:
#get the number of rows after grouping
taxi_df.shape

(1745586, 3)

In [165]:
# change the transit_timestamp into a datetime object
taxi_df['transit_timestamp'] = pd.to_datetime(taxi_df['transit_timestamp'])

In [166]:
#change zone_id to categorical type
taxi_df['zone_id'] = taxi_df['zone_id'].astype('int')

In [167]:
taxi_df.dtypes

transit_timestamp    datetime64[ns]
zone_id                       int64
passenger_count               int64
dtype: object

In [168]:
##group the subway df by the combo of timestamp and zone_id summing up the ridership values.
taxi_df = taxi_df.groupby(['transit_timestamp', 'zone_id']).sum().reset_index()

In [169]:
#count the number of duplicate rows
taxi_df.duplicated().sum()

0

In [170]:
#what is the max and min dates in the df
print(taxi_df['transit_timestamp'].min())
print(taxi_df['transit_timestamp'].max())

2021-01-01 00:00:00
2024-04-01 00:00:00


In [171]:
# Generate a complete range of hourly timestamps from start to end
start_timestamp = taxi_df['transit_timestamp'].min()
end_timestamp = taxi_df['transit_timestamp'].max()
complete_range = pd.date_range(start=start_timestamp, end=end_timestamp, freq='H')

# Identify missing timestamps
existing_timestamps = taxi_df['transit_timestamp']
missing_timestamps = complete_range.difference(existing_timestamps)

print(f"Length of Missing Timestamps:\n{len(missing_timestamps)}")

Length of Missing Timestamps:
0


In [125]:
subway_df = pd.read_csv('cleaned-subway-data-1.csv')

subway_df.head()

,transit_timestamp,station_complex_id,station_complex,fare_class_category,ridership,latitude,longitude
0,2024-04-25 17:00:00,222,Roosevelt Island (F),Metrocard - Seniors & Disability,21,40.7591,-73.9533
1,2024-04-25 06:00:00,310,"96 St (1,2,3)",Metrocard - Full Fare,49,40.7939,-73.9723
2,2024-04-25 10:00:00,313,"72 St (1,2,3)",Metrocard - Unlimited 7-Day,90,40.7785,-73.9820
3,2024-04-25 12:00:00,329,Rector St (1),OMNY - Seniors & Disability,2,40.7075,-74.0138
4,2024-04-25 16:00:00,146,181 St (A),OMNY - Seniors & Disability,7,40.8517,-73.9380


In [126]:
#drop all columns except of transit_timestamp, latitude, longitude and risership

subway_df = subway_df[['transit_timestamp', 'latitude', 'longitude', 'ridership']]

subway_df.head()

,transit_timestamp,latitude,longitude,ridership
0,2024-04-25 17:00:00,40.7591,-73.9533,21
1,2024-04-25 06:00:00,40.7939,-73.9723,49
2,2024-04-25 10:00:00,40.7785,-73.9820,90
3,2024-04-25 12:00:00,40.7075,-74.0138,2
4,2024-04-25 16:00:00,40.8517,-73.9380,7


In [127]:
# change the transit_timestamp into a datetime object
subway_df['transit_timestamp'] = pd.to_datetime(subway_df['transit_timestamp'])

In [128]:
subway_df.dtypes

transit_timestamp    datetime64[ns]
latitude                    float64
longitude                   float64
ridership                     int64
dtype: object

In [129]:
#what is the max and min dates in the df
print(subway_df['transit_timestamp'].min())
print(subway_df['transit_timestamp'].max())

2022-02-01 00:00:00
2024-05-23 23:00:00


In [130]:
# Generate a complete range of hourly timestamps from start to end
start_timestamp = subway_df['transit_timestamp'].min()
end_timestamp = subway_df['transit_timestamp'].max()
complete_range = pd.date_range(start=start_timestamp, end=end_timestamp, freq='H')

# Identify missing timestamps
existing_timestamps = subway_df['transit_timestamp']
missing_timestamps = complete_range.difference(existing_timestamps)

print(f"Length of Missing Timestamps:\n{len(missing_timestamps)}")
missing_timestamps

Length of Missing Timestamps:
5


DatetimeIndex(['2022-03-13 02:00:00', '2022-12-31 03:00:00',
               '2022-12-31 04:00:00', '2023-03-12 02:00:00',
               '2024-03-10 02:00:00'],
              dtype='datetime64[ns]', freq=None)

In [172]:
#keep rows with time stamps that are on or before 2024-04-01 00:00:00 in the subway dataset.
subway_df = subway_df[subway_df['transit_timestamp'] <= '2024-04-01 00:00:00']

In [173]:
#keep rows with time stamps that are on or after 2022-02-01 00:00:00 in the taxi dataset.
taxi_df = taxi_df[taxi_df['transit_timestamp'] >= '2022-02-01 00:00:00']

In [132]:
#assign taxi zones to subway locations.
# Load GeoJSON data into a GeoDataFrame
def load_geojson_gpd(filepath):
    return gpd.read_file(filepath)

# Function to find zones using spatial join in geopandas
def assign_zones(df, gdf):
    # Convert DataFrame to GeoDataFrame
    gdf_points = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
    gdf_points.set_crs(gdf.crs, inplace=True)  # Ensure CRS matches if known; otherwise, assume it matches

    # Spatial join points to polygons
    joined = gpd.sjoin(gdf_points, gdf, how="left", op='within')
    return joined['location_id']

# Load GeoJSON data into a GeoDataFrame
geo_df = load_geojson_gpd('/content/NYC Taxi Zones.geojson')


# Assign zones using the efficient spatial join
subway_df['zone_id'] = assign_zones(subway_df, geo_df)

subway_df.head()

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


,transit_timestamp,latitude,longitude,ridership,zone_id
265,2024-02-28 14:00:00,40.7104,-74.0066,145,209
707,2024-02-14 00:00:00,40.7031,-74.0130,32,88
4708,2024-03-16 05:00:00,40.7382,-73.9962,7,234
6123,2024-03-16 11:00:00,40.7195,-73.9999,132,144
6906,2024-03-30 18:00:00,40.7986,-73.9416,53,74


In [133]:
#drop the latitude and longitude columns from the subway df
subway_df = subway_df.drop(['latitude', 'longitude'], axis=1)

In [134]:
#group the subway df by the combo of timestamp and zone_id summing up the ridership values.
subway_df = subway_df.groupby(['transit_timestamp', 'zone_id']).sum().reset_index()

In [135]:
#count the number of duplicate rows
subway_df.duplicated().sum()

0

In [136]:
#change zone_id to int type
subway_df['zone_id'] = subway_df['zone_id'].astype('int')

In [152]:
subway_df.dtypes

transit_timestamp    datetime64[ns]
zone_id                       int64
ridership                   float64
dtype: object

In [138]:
#preview the dataframe.
subway_df.head()

,transit_timestamp,zone_id,ridership
0,2022-02-01,113,101
1,2022-02-01,114,292
2,2022-02-01,116,50
3,2022-02-01,125,71
4,2022-02-01,127,34


In [139]:
# Get unique zone IDs
unique_zone_ids = subway_df['zone_id'].unique()

# Get unique timestamps
unique_timestamps = subway_df['transit_timestamp'].unique()

# Create a DataFrame with all combinations of timestamps and zone IDs
all_combinations = pd.DataFrame(list(itertools.product(unique_timestamps, unique_zone_ids)), columns=['transit_timestamp', 'zone_id'])

# Merge with the original DataFrame
subway_df = pd.merge(all_combinations, subway_df, on=['transit_timestamp', 'zone_id'], how='left')

# Fill NaN values in the ridership column with zero
subway_df['ridership'].fillna(0, inplace=True)

In [154]:
subway_df.head()

,transit_timestamp,zone_id,ridership
0,2022-02-01,113,101.0
1,2022-02-01,114,292.0
2,2022-02-01,116,50.0
3,2022-02-01,125,71.0
4,2022-02-01,127,34.0


In [155]:
#change ridership type to int
subway_df['ridership'] = subway_df['ridership'].astype('int')

In [156]:
#get the row in the subway_df for which timestamp is 2022-02-01	midnight and zone_id is 100
# Filter the row in subway_df for the given timestamp and zone_id
filtered_row = subway_df[(subway_df['transit_timestamp'] == '2022-02-01 00:00:00') & (subway_df['zone_id'] == 100) ]

print(filtered_row)

   transit_timestamp  zone_id  ridership
51        2022-02-01      100          0


In [157]:
#count the number of unique zone id in the subway df
subway_df['zone_id'].nunique()

53

In [158]:
#count the number of unique zone id in the taxi df
taxi_df['zone_id'].nunique()

67

In [174]:
# combine the subway and taxi dfs into a single df called df. remove duplicate columns.
df = pd.merge(taxi_df, subway_df,on=['transit_timestamp', 'zone_id'], how='left')
df = df.drop_duplicates()
df.head()

,transit_timestamp,zone_id,passenger_count,ridership
0,2022-02-01,4,20,NaN
1,2022-02-01,12,1,NaN
2,2022-02-01,13,23,NaN
3,2022-02-01,24,20,25.0
4,2022-02-01,41,32,154.0


In [175]:
#get the zone ids that are in the taxi_df and not in the subway df
zone_ids_without_subway = set(taxi_df['zone_id']) - set(subway_df['zone_id'])
zone_ids_without_subway

{4, 12, 13, 50, 105, 120, 128, 137, 140, 158, 170, 194, 233, 262}

In [176]:
#add a column called subway_station_present in the df with the value of False if the zone id is in the zone_ids_without_subway set
df['subway_station_present'] = ~df['zone_id'].isin(zone_ids_without_subway)

In [180]:
df.rename(columns={'passenger_count': 'taxi_ridership', 'ridership': 'subway_ridership' }, inplace=True)

In [181]:
df.head(20)

,transit_timestamp,zone_id,taxi_ridership,subway_ridership,subway_station_present
0,2022-02-01,4,20,NaN,False
1,2022-02-01,12,1,NaN,False
2,2022-02-01,13,23,NaN,False
3,2022-02-01,24,20,25.0,True
4,2022-02-01,41,32,154.0,True
5,2022-02-01,42,26,46.0,True
6,2022-02-01,43,29,92.0,True
7,2022-02-01,45,8,61.0,True
8,2022-02-01,48,171,1.0,True
9,2022-02-01,50,52,NaN,False


In [178]:
df.dtypes

transit_timestamp         datetime64[ns]
zone_id                            int64
passenger_count                    int64
ridership                        float64
subway_station_present              bool
dtype: object

In [179]:
df.shape

(1167176, 5)

In [182]:
#percentage of NaN values in the df (those correspond to zones where there is not any subway station in!)
print(f"percentage of NaN values in the ridership column is {(df.isna().sum() / df.shape[0]) * 100}")

percentage of NaN values in the ridership column is transit_timestamp          0.000000
zone_id                    0.000000
taxi_ridership             0.000000
subway_ridership          16.849815
subway_station_present     0.000000
dtype: float64


In [183]:
#percentage of 0 values in the subway_ridership column (those correspond to times when no subway data was recorded!)
print(f"percentage of 0 values in the subway_ridership column is {(df[df['subway_ridership'] == 0].shape[0] / df.shape[0]) * 100}")


percentage of 0 values in the subway_ridership column is 2.289029246660315


In [185]:
#count the number of unique zone id
df['zone_id'].nunique()

67

In [188]:
# add zero instead of NaN in the subway_ridership column.
# zero could mean that there is no subway station in that zone, or that there was no data recorded for that time. the distinction is still in the dataset using the subway_station_present column!
df['subway_ridership'] = df['subway_ridership'].fillna(0)

In [189]:
#change total_ridership and subway ridership type to int
df['subway_ridership'] = df['subway_ridership'].astype('int')

In [190]:
#preview the dataframe
df.head()

,transit_timestamp,zone_id,taxi_ridership,subway_ridership,subway_station_present
0,2022-02-01,4,20,0,False
1,2022-02-01,12,1,0,False
2,2022-02-01,13,23,0,False
3,2022-02-01,24,20,25,True
4,2022-02-01,41,32,154,True


In [191]:
#count the number of rows with duplicate timestamp and zone_id combo
df[['transit_timestamp','zone_id']].duplicated().sum()

0

In [201]:
# Function to calculate the area of each zone and store it in a dictionary
def calculate_zone_areas(gdf):
    # Ensure the GeoDataFrame has a coordinate reference system set
    if gdf.crs is None:
        raise ValueError("GeoDataFrame must have a coordinate reference system (CRS) set.")

    # Reproject to a suitable projected CRS (e.g., UTM)
    gdf = gdf.to_crs(epsg=32633)  # Example EPSG code for UTM zone 33N

    # Calculate the area of each polygon in square meters
    gdf['area'] = gdf.geometry.area

    # Create a dictionary with location_id as keys and areas as values
    area_dict = gdf.set_index(gdf['location_id'].astype(int))['area'].to_dict()

    return area_dict

# Example usage
# Load GeoJSON file into GeoDataFrame
gdf = load_geojson_gpd('/content/NYC Taxi Zones.geojson')

# Calculate the area of each zone and store in a dictionary
zone_areas_m2 = calculate_zone_areas(gdf)

# Print the resulting dictionary
print(zone_areas_m2)



{1: 17166205.262391683, 2: 31340334.066099223, 3: 6832547.1469850475, 4: 1740160.713626942, 5: 11010612.833079645, 6: 8922379.504218347, 7: 8518903.876762534, 8: 580540.1344511067, 9: 7400109.928131098, 24: 914547.2496660567, 10: 9567071.564394284, 11: 4475155.882783008, 12: 236505.43818529288, 13: 1068198.795442277, 18: 3234224.8882483165, 25: 2725177.4418478687, 14: 14614813.05067437, 15: 10014923.954035757, 22: 10166739.180006525, 23: 46094384.861298464, 16: 17492535.97972123, 17: 7085608.40548386, 19: 11962259.886989849, 20: 2924269.9376217513, 21: 7797320.312202264, 26: 11756054.752283955, 27: 15955882.555162538, 28: 6381100.701122075, 33: 2392208.4291282524, 29: 3747464.59358529, 31: 7260494.637447402, 32: 3278895.308444962, 30: 3215264.729796244, 34: 2429667.6838337635, 35: 7115122.523447807, 36: 5430899.153897927, 37: 9916031.973651487, 38: 7180360.645584411, 39: 17858795.013051298, 40: 2392024.437526738, 41: 3120088.6284180954, 45: 1616560.7925335704, 46: 3743842.6972154425, 4

In [204]:
#add the zone area to each zone id as a new column
df['zone_area_m2'] = df['zone_id'].map(zone_areas_m2)

In [208]:
df.head()

,transit_timestamp,zone_id,taxi_ridership,subway_ridership,subway_station_present,zone_area_m2
0,2022-02-01,4,20,0,False,1.740161e+06
1,2022-02-01,12,1,0,False,2.365054e+05
2,2022-02-01,13,23,0,False,1.068199e+06
3,2022-02-01,24,20,25,True,9.145472e+05
4,2022-02-01,41,32,154,True,3.120089e+06


In [209]:
#export the dataframe to a csv file
df.to_csv('taxi_subway_zones_df.csv', index=False)